In [1]:
import camelot
import pandas as pd
import openpyxl
import os
import pdfplumber
import re
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [2]:
columns = {                                                                                                   
  "impactGWP100_kgCO2e": "(^Global warming potential)|(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^Ozone depletion potential)|(^ODP$)|(^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^Acidification potential)|(^AP$)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^Photochemical ozone creation potential)|(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^Abiotic depletion potential for non fossil \nresources$)|(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^Abiotic depletion potential for fossil \nresources$)|(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^Förnybar \nprimärenergi Energiresurs)|(^PERE$)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^Förnybar \nprimärenergi Råmaterial)|(^PERM$)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^Förnybar \nprimärenergi TOTAL)|(^PERT$)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^Icke-förnybar \nprimärenergi Energiresurs)|(^Non\xadrenewable primary energy as energy carrier$)|(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^Icke-förnybar \nprimärenergi Råmaterial)|(^Non\xadrenewable primary energy as material utilization$)|(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^Icke-förnybar \nprimärenergi TOTAL)|(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
  
  "recyclingMaterialUse_kg": "(^Sekundära material)|(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^Förnybara sekundära bränslen)|(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^Icke-förnybara sekundära bränslen)|(^Use of non\xadrenewable secondary fuels$)|(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^Vattenanvändning)|(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
  "wasteHazardous_kg": "(^Farligt avfall)|(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^Icke-farligt avfall)|(^Non\xadhazardous waste disposed$)|(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^Radioaktivt avfall)|(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^Komponenter för återanvändning)|(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
  "recyclableMaterialsOutput_kg": "(^Material för återvinning)|(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^Material för energiåtervinning)|(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^Exporterad energi)|(^EE$)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

In [3]:
def epd_number(file_name):
    with pdfplumber.open('./New/' + file_name) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        #print(text)
        item = re.findall('EPD registration number.*', text)
        try:
            r = item[0].split(' ')[3]
        except:
            r = ''
        
    return r

In [4]:

def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'Envirodec Group 4.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = epd_number(file_data["file_name"])
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-12-13/Envirodec/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    if header in ['wasteHazardous_kg','wasteNonHazardous_kg','wasteRadioactive_kg','reusableMaterialsOutput_kg',
                                  'recyclableMaterialsOutput_kg','energyMaterialsOutput_kg','exportedEnergyOutput_MJ_thermal',
                                  'exportedEnergyOutput_MJ_electric','exportedEnergyOutput_MJ','impactGWP100_kgCO2e',
                                  'impactODP_kgCFC11e','impactAP_kgSO2e','impactEP_kgPO4e','impactPOCP_kgEthenee',
                                  'impactADPElements_kgSbe','impactADPFossilFuels_MJ']:
                        num_of_columns = len(df.columns)                    
                        i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                        sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                        replaced = df[df[0].str.contains(columns[header])][i - 1].to_list()[0].replace(',', '.')
                    else:
                        num_of_columns = len(df.columns)                    
                        i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                        sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                        replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            try:
                                value = float(replaced.replace('\xad', '-'))
                            except:
                                value = float(replaced.replace('\n', ''))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [5]:
def clean_df(df):
    transformed = df.applymap(lambda x: x.split(' ') if ('E-' in x) or ('E+' in x) or (x == '') else x)
    for i in range(transformed.shape[0]):
        row = transformed.iloc[[i]]
        for index, element in row.iloc[0].items():
            if element == ['']:
                for index_s, search in row.iloc[0].items():
                    if (len(search) > 1) and ('[kg' not in search) and (type(search) == list):
                        if index < index_s:        
                            transformed.loc[i,index] = [search[0]]
                            row.iloc[0][index] = [search[0]]
                            del search[0]
                        else: 
                            transformed.loc[i,index] = [search[1]]                            
                            row.iloc[0][index] = [search[1]]
                            del search[1]
                        break
    return transformed.applymap(lambda x: x[0] if type(x) == list else x)



def clean_df2(df):
    df[0][2] = df[0][1] + ' ' + df[1][2]
    df[0][3] = df[0][1] + ' ' + df[1][3]

    df[0][5] = df[0][4] + ' ' + df[1][5]
    df[0][6] = df[0][4] + ' ' + df[1][6]

    df[0][1] = df[0][1] + ' ' + df[1][1]
    df[0][4] = df[0][4] + ' ' + df[1][4]
    #del df[1]
    return df 


In [8]:
files = os.listdir('./Group 2')
# files = ['15230.pdf', '14983.pdf.pdf', '14956.pdf']
r = []
for file in files:
    print("************ FILE NAME: " + file + " ****************")
    tables = camelot.read_pdf('./Group 2/' + file, pages='all')
    dataFrames = []

    file_info = {}
    stages = []
    for table in tables:
        df = table.df
        stages_added = False
        #stages = ['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
        #stages = ['A1-A3', 'A4', 'A1-A4']
        gwp = ['Global warming potential (GWP)','GWP-total', 'Global warming potential \xad total', 'Global warming potential', 'GWP', 'Globales Erwärmungspotenzial ­ total', 'Globales Erwärmungspotenzial']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:

                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                print(1)
                dataFrames.append(df)


        pere = ['Förnybar \nprimärenergi', 'PERE', 'Renewable primary energy as energy carrier', 'Erneuerbare Primärenergie als Energieträger']
        #pere = ['Förnybar \nprimärenergi']
        for p in pere:
            if p in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                
                dataFrames.append(df)
                print(2)

        hwd = ['HWD', 'Hazardous waste disposed', 'Gefährlicher Abfall zur Deponie']
        #hwd = ['Farligt avfall', 'Komponenter för återanvändning']
        for h in hwd:
            if h in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(3)

                
    result = pd.concat(dataFrames)

    file_info["dataFrame"] = result

        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    

make_excel_file(r)


#r

************ FILE NAME: epd2234.pdf ****************
2
3
************ FILE NAME: epd2235.pdf ****************
2
3
************ FILE NAME: epd2236.pdf ****************
2
3
epd2234.pdf
[]
*******************
epd2235.pdf
[]
*******************
epd2236.pdf
[]
*******************


In [14]:
tables = camelot.read_pdf('./Group 1/epd4436.pdf', pages='all')
print(len(tables))

11


In [45]:
tables[4].df

,0,1,2,3,4,5
0,PARAMETER,,ENHET,A1-A3,A4,A1-A4*
1,Förnybar \nprimärenergi,Energiresurs,"MJ, \nnettovärmevärde","1,75E+02","7,68E+01","2,51E+02"
2,,Råmaterial,"MJ, \nnettovärmevärde",0,0,0
3,,TOTAL,"MJ, \nnettovärmevärde","1,75E+02","7,68E+01","2,51E+02"
4,Icke-förnybar \nprimärenergi,Energiresurs,"MJ, \nnettovärmevärde","7,01E+02","1,83E+02","8,84E+02"
5,,Råmaterial,"MJ, \nnettovärmevärde",0,0,0
6,,TOTAL,"MJ, \nnettovärmevärde","7,01E+02","1,83E+02","8,84E+02"
7,Sekundära material,,kg,"2,60E+01",0,"2,60E+01"
8,Förnybara sekundära bränslen,,"MJ, \nnettovärmevärde","2,77E+01",0,"2,77E+01"
9,Icke-förnybara sekundära bränslen,,"MJ, \nnettovärmevärde","6,23E+01",0,"6,23E+01"


In [17]:
header = 'wasteHazardous_kg'
if header in ['wasteHazardous_kg','wasteNonHazardous_kg','wasteRadioactive_kg','reusableMaterialsOutput_kg',
              'recyclableMaterialsOutput_kg','energyMaterialsOutput_kg','exportedEnergyOutput_MJ_thermal',
              'exportedEnergyOutput_MJ_electric','exportedEnergyOutput_MJ','impactGWP100_kgCO2e',
              'impactODP_kgCFC11e','impactAP_kgSO2e','impactEP_kgPO4e','impactPOCP_kgEthenee',
              'impactADPElements_kgSbe','impactADPFossilFuels_MJ']:
    print('a')

a
